In [1]:
# ! pip install hazm
# ! pip install gensim
# ! pip -q install clean-text[gpl]

In [2]:
import pandas as pd
import numpy as np
import re
from __future__ import unicode_literals
import hazm
import nltk
import codecs
import tqdm
import gensim
from cleantext import clean

### Load Data

In [3]:
with open('masnavi.txt', 'r', encoding="utf8") as infile:
    masnavi_file = infile.readlines()

In [4]:
stopwords = [x.strip() for x in codecs.open('stopwords.txt','r','utf-8').readlines()]

In [5]:
persian_punctuation = ['،','؟',':','\*','«',"»"]

In [6]:
normalizer = hazm.Normalizer()

In [7]:
lemmatizer = hazm.Lemmatizer()

In [8]:
# Store Masnavi
masnavi = []

In [9]:
def clean_text(text, tokenize = False):
    text = normalizer.normalize(text)
    text = lemmatizer.lemmatize(text)
    text = re.sub(r"|".join(persian_punctuation), " ", text)
    regex = r"\b(?:" + "|".join(map(re.escape, stopwords)) + r")\b"
    text = re.sub(regex, " ", text)
    text = re.sub(u"\u200c" , "", text)
    text = re.sub(r'\s+', " ", text)
    text = text.strip()
    return text

In [10]:
def process_couplet(text):
    result = re.search("(\d{1,3})\.(\d{1,3})", text)
    # check if line contains a couplet
    if result:
        pno, cno = result.groups()
        # delete Daftar and Poem number
        couplet = re.sub("(\d{1,3})\.(\d{1,3})", "", text)
        # extract mesra
        hemistich = couplet.split("\t")[1:3]
        # clean mesra
        cleaned_hemistich = [clean_text(h) for h in hemistich]
        return pno, cno, "\t".join(hemistich), " ".join(cleaned_hemistich), cleaned_hemistich[0], cleaned_hemistich[1]
    return None

In [11]:
daftar = 0
for couplet in masnavi_file:
    if re.search("^(?:دفتر).*.(?:مثنوی)$", couplet):
        daftar += 1
        if daftar == 7:
            break
        print(f"Processing Daftar {daftar}")
    else:
        process_result = process_couplet(couplet)
        if process_result:
            pno = process_result[0]
            cno = process_result[1]
            c = process_result[2]
            cc = process_result[3]
            h1 = process_result[4]
            h2 = process_result[5]
            masnavi.append((daftar, pno, cno, c, cc, h1, h2))

Processing Daftar 1
Processing Daftar 2
Processing Daftar 3
Processing Daftar 4
Processing Daftar 5
Processing Daftar 6


In [12]:
masnavi_df = pd.DataFrame(masnavi, columns=['Daftar', 'Poem', 'CNo', 'Couplet', 'CCouplet', 'Hemistich1', 'Hemistich2'])

In [14]:
masnavi_df.Hemistich2[200:250]

200           جفته انداخت زخم
201            حاذقی مرکز تند
202                عاقلی خاری
203            دست میزد آزمود
204              پرسید دوستان
205      مقام خواجگان شهر تاش
206      نبض جستنش میداشت هوش
207           مقصود جانش جهان
208                       شهر
209         کدامین شهر میبودی
210              رنگ نبض نگشت
211                   نان نمک
212    نی رگش جنبید نی رخ زرد
213         بپرسید سمرقند قند
214          آب چشمش روان جوی
215      خواجه زرگر شهرم خرید
216      بگفت زآتش غم برفروخت
217              سمرقندی زرگر
218               اصل درد بلا
219              پل کوی غاتفر
220           کنیزک رستی عذاب
221          علاجت سحرها نمود
222                 باران چمن
223              مشفق ترم پدر
224                 شاه جستجو
225                     زنهار
226                مرادت حاصل
227                  مراد جفت
228                سبزی بستان
229             پرورش یافتندی
230            رنجور ایمن بیم
231           وعده مجازی تاسه
232       وعدۀ نااهل رنج روان
233       

### Process dataframe data types

In [15]:
masnavi_df['Daftar'] = masnavi_df['Daftar'].apply(pd.to_numeric)
masnavi_df['Poem'] = masnavi_df['Poem'].apply(pd.to_numeric)
masnavi_df['CNo'] = masnavi_df['CNo'].apply(pd.to_numeric)

In [16]:
masnavi_df[masnavi_df['Daftar']==1]

,Daftar,Poem,CNo,Couplet,CCouplet,Hemistich1,Hemistich2
0,1,1,1,بشنو از نی، چون حكایت میكند\tواز جدائی ها شكای...,بشنو نی جدائی شکایت,بشنو نی,جدائی شکایت
1,1,1,2,کز نیستان تا مرا ببریده اند\tاز نفیرم مرد و زن...,نیستان ببریده نفیرم مرد زن نالیده,نیستان ببریده,نفیرم مرد زن نالیده
2,1,1,3,سینه خواهم شرحه شرحه از فراق\tتا بگویم شرح درد...,سینه شرحه شرحه فراق شرح درد اشتیاق,سینه شرحه شرحه فراق,شرح درد اشتیاق
3,1,1,4,هر كسی كاو دور ماند از اصل ِ خویش\tباز جوید رو...,اصل جوید روزگار وصل,اصل,جوید روزگار وصل
4,1,1,5,من به هر جمعیتی نالان شدم\tجفت بَد حالان و خوش...,جمعیتی نالان جفت حالان حالان,جمعیتی نالان,جفت حالان حالان
...,...,...,...,...,...,...,...
4530,1,182,10,بر همان بو می خوری این خشك را\tبعد از آن كامیخ...,بو خشک کامیخت معنی ثری,بو خشک,کامیخت معنی ثری
4531,1,182,11,گشت خاك آمیز و خشك و گوشت بُر\tز آن گیاه اكنون...,خاک آمیز خشک گوشت گیاه بپرهیز شتر,خاک آمیز خشک گوشت,گیاه بپرهیز شتر
4532,1,182,12,سخت خاك آلود می آید سُخُن\tآب تیره شد، سر چه ب...,خاک آلود سخن آب تیره بند,خاک آلود سخن,آب تیره بند
4533,1,182,13,تا خدایش باز صاف و خوش كند\tآنكه تیره كرد هم ص...,خدایش صاف تیره صافش,خدایش صاف,تیره صافش


### Create Tokenized columns

In [17]:
masnavi_df['Couplet_tokenized'] = masnavi_df['CCouplet'].apply(lambda x:hazm.word_tokenize(x))

In [18]:
masnavi_df['Hemistich1_tokenized'] =  masnavi_df['Hemistich1'].apply(lambda x:hazm.word_tokenize(x))

In [19]:
masnavi_df['Hemistich2_tokenized'] =  masnavi_df['Hemistich2'].apply(lambda x:hazm.word_tokenize(x))

In [20]:
masnavi_df

,Daftar,Poem,CNo,Couplet,CCouplet,Hemistich1,Hemistich2,Couplet_tokenized,Hemistich1_tokenized,Hemistich2_tokenized
0,1,1,1,بشنو از نی، چون حكایت میكند\tواز جدائی ها شكای...,بشنو نی جدائی شکایت,بشنو نی,جدائی شکایت,"[بشنو, نی, جدائی, شکایت]","[بشنو, نی]","[جدائی, شکایت]"
1,1,1,2,کز نیستان تا مرا ببریده اند\tاز نفیرم مرد و زن...,نیستان ببریده نفیرم مرد زن نالیده,نیستان ببریده,نفیرم مرد زن نالیده,"[نیستان, ببریده, نفیرم, مرد, زن, نالیده]","[نیستان, ببریده]","[نفیرم, مرد, زن, نالیده]"
2,1,1,3,سینه خواهم شرحه شرحه از فراق\tتا بگویم شرح درد...,سینه شرحه شرحه فراق شرح درد اشتیاق,سینه شرحه شرحه فراق,شرح درد اشتیاق,"[سینه, شرحه, شرحه, فراق, شرح, درد, اشتیاق]","[سینه, شرحه, شرحه, فراق]","[شرح, درد, اشتیاق]"
3,1,1,4,هر كسی كاو دور ماند از اصل ِ خویش\tباز جوید رو...,اصل جوید روزگار وصل,اصل,جوید روزگار وصل,"[اصل, جوید, روزگار, وصل]",[اصل],"[جوید, روزگار, وصل]"
4,1,1,5,من به هر جمعیتی نالان شدم\tجفت بَد حالان و خوش...,جمعیتی نالان جفت حالان حالان,جمعیتی نالان,جفت حالان حالان,"[جمعیتی, نالان, جفت, حالان, حالان]","[جمعیتی, نالان]","[جفت, حالان, حالان]"
...,...,...,...,...,...,...,...,...,...,...
27948,6,145,49,قصه کوته کن، که رفتم در حجاب\tهین خمش والله اع...,قصه کوته رفتم حجاب هین خمش والله اعلم بالصواب,قصه کوته رفتم حجاب,هین خمش والله اعلم بالصواب,"[قصه, کوته, رفتم, حجاب, هین, خمش, والله, اعلم,...","[قصه, کوته, رفتم, حجاب]","[هین, خمش, والله, اعلم, بالصواب]"
27949,6,145,50,* شکر کاین نامه به عنوانی رسید\tگم نشد نقد و ب...,شکر نامه عنوانی گم نقد اخوانی,شکر نامه عنوانی,گم نقد اخوانی,"[شکر, نامه, عنوانی, گم, نقد, اخوانی]","[شکر, نامه, عنوانی]","[گم, نقد, اخوانی]"
27950,6,145,51,* نردبان آسمان است این کلام\tهر که از این بر ر...,نردبان آسمان کلام بام,نردبان آسمان کلام,بام,"[نردبان, آسمان, کلام, بام]","[نردبان, آسمان, کلام]",[بام]
27951,6,145,52,* نه به بام چرخ کان اخضر بود\tبل به بامی کز فل...,بام چرخ اخضر بل بامی فلک برتر,بام چرخ اخضر,بل بامی فلک برتر,"[بام, چرخ, اخضر, بل, بامی, فلک, برتر]","[بام, چرخ, اخضر]","[بل, بامی, فلک, برتر]"


### Frequency Analysis

In [21]:
from itertools import chain
from collections import Counter

In [22]:
all_words = list(chain.from_iterable(masnavi_df.Couplet_tokenized))

In [23]:
words_frequencies = nltk.FreqDist(all_words).most_common(100)

In [24]:
words_frequencies[:10]

[('جان', 1538),
 ('دل', 1047),
 ('آب', 941),
 ('نی', 911),
 ('دست', 776),
 ('چشم', 731),
 ('نور', 648),
 ('جهان', 602),
 ('عقل', 565),
 ('تن', 495)]

In [25]:
print ('%-16s' % 'Number of words', '%-16s' % len(all_words))
print ('%-16s' % 'Number of unique words', '%-16s' % len(set(all_words)))
avg=np.sum([len(word) for word in all_words])/len(all_words)
print ('%-16s' % 'Average word length', '%-16s' % avg)
print ('%-16s' % 'Longest word', '%-16s' % all_words[np.argmax([len(word) for word in all_words])])

Number of words  165846          
Number of unique words 22641           
Average word length 3.9938497160015918
Longest word     استخوانهاشان    


### TF/IDF Analysis

In [26]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

In [27]:
cv=CountVectorizer()
word_count_vector = cv.fit_transform(all_words)

In [28]:
tfidf_transformer = TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [29]:
# print idf values 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names_out(),columns=["idf_weights"]) 
 
# sort ascending 
df_idf = df_idf.sort_values(by=['idf_weights'])

In [64]:
df_idf.to_csv('idf.csv')

In [52]:
important_words = list(df_idf[-10000:].index)

In [53]:
important_words

['الصدقة',
 'الصدقات',
 'الصدق',
 'وابریم',
 'الصدف',
 'دیوشان',
 'نکات',
 'نکاحی',
 'نکاری',
 'وابرید',
 'وابری',
 'نکاشت',
 'وابرد',
 'الصدر',
 'وآنجزا',
 'دوریش',
 'نکردندت',
 'دیوانند',
 'الشقی',
 'الشفی',
 'نکوهیدی',
 'نکویست',
 'وآنرا',
 'نکوبی',
 'وآندیگر',
 'خبیثین',
 'نکیر',
 'الشفق',
 'خالفوا',
 'وآنجهان',
 'نگارستان',
 'نکویم',
 'نکردند',
 'الشهب',
 'نکوئیهای',
 'دوریم',
 'نکردید',
 'نکردیم',
 'خالئی',
 'الأبیات',
 'نکشت',
 'همجنسند',
 'نکشتش',
 'الأرزاق',
 'وآنچنانکه',
 'نکفت',
 'الشکور',
 'خالئیم',
 'وآنچنان',
 'دیوانگانه',
 'هیزان',
 'نگشتند',
 'نگشته',
 'نیامیزم',
 'نیامیزند',
 'الاحمق',
 'نیاوردند',
 'الدوا',
 'الدلیل',
 'خبرهات',
 'دوزید',
 'الدرة',
 'هوشهاست',
 'نیایندش',
 'الدر',
 'الدخان',
 'نیتش',
 'نیایدشان',
 'نیتها',
 'نیامیزد',
 'نیاموزید',
 'دیرتر',
 'هولم',
 'نیازارد',
 'دیدیمش',
 'هولش',
 'نیاسایند',
 'هوشیم',
 'نیاشامد',
 'نیام',
 'دوزنده',
 'الدولتین',
 'نیاموزد',
 'نیاموزم',
 'نیاموزی',
 'الذات',
 'الذنوب',
 'الدجی',
 'الدار',
 'الارباح',
 'هوستان',
 'الا

### Topic Modeling

In [54]:
from gensim.test.utils import datapath
from pprint import pprint

In [55]:
dp_groups = masnavi_df.groupby(['Daftar'])

In [56]:
poems = []
for name, group in dp_groups:
    poems.append([t for l in group['Couplet_tokenized'] for t in l if t in important_words])

In [57]:
dictionary = gensim.corpora.Dictionary(poems)

In [58]:
bow_corpus = [dictionary.doc2bow(doc) for doc in masnavi_df.Couplet_tokenized]

In [59]:
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 30, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [60]:
lda_path = datapath("model")
lda_model.save(lda_path)

In [61]:
lda = lda_model.load(lda_path)

In [62]:
lda.print_topics()

[(24,
  '0.003*"تولاه" + 0.003*"أینما" + 0.003*"عددا" + 0.003*"صعدا" + 0.003*"شیننا" + 0.003*"زبننا" + 0.003*"خذ" + 0.003*"میزنش" + 0.003*"سوفسطایی" + 0.003*"ناصحانشان"'),
 (18,
  '0.003*"نازکم" + 0.003*"نغنوده" + 0.003*"کاشتید" + 0.003*"بگذاشتید" + 0.002*"تنگت" + 0.002*"آدابی" + 0.002*"ترتیبی" + 0.002*"جهلان" + 0.002*"جسر" + 0.002*"منظری"'),
 (17,
  '0.003*"یطلب" + 0.003*"الإنسان" + 0.003*"أنکر" + 0.003*"فإذا" + 0.003*"أحسنت" + 0.003*"الیه" + 0.003*"اتق" + 0.003*"خزم" + 0.003*"پزم" + 0.002*"دستکت"'),
 (21,
  '0.003*"نشاطی" + 0.003*"ملتویست" + 0.003*"شغالک" + 0.003*"آغازتان" + 0.003*"وانگویم" + 0.003*"رازتان" + 0.002*"اجهد" + 0.002*"اعتبر" + 0.002*"تصب" + 0.002*"فانتبه"'),
 (29,
  '0.003*"خفتنش" + 0.003*"چشمانم" + 0.002*"تصغیر" + 0.002*"تحقیر" + 0.002*"طفلکم" + 0.002*"ستونهای" + 0.002*"مرضهای" + 0.002*"خللهای" + 0.002*"دریاش" + 0.002*"کریمم"'),
 (8,
  '0.003*"جبل" + 0.003*"صار" + 0.003*"رأیتم" + 0.003*"دانائیش" + 0.003*"سالکانش" + 0.003*"سوزنگر" + 0.003*"ساکنانش" + 0.003*"سلامیشان" + 0